In [1]:
#default_exp qdrant

In [2]:
#export
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
from tqdm import tqdm

from qdrant_client import QdrantClient
import qdrant_client
collection_name = "food_images"

# collection_name = "food_categories"



dev


##### production
docker pull qdrant/qdrant:master
docker run -p 6335:6333 \
    -v $(pwd)/qdrant_prod_2206_storage:/qdrant/storage \
    --name qdrant_prod_2206 \
    qdrant/qdrant
docker update --restart unless-stopped qdrant_prod_2206

##### dev
docker pull qdrant/qdrant:master
docker run -p 6334:6333 \
    -v $(pwd)/qdrant_storage_dev:/qdrant/storage \
    --name qdrant_dev \
    qdrant/qdrant
docker update --restart unless-stopped qdrant_dev

In [3]:
!pip install --upgrade qdrant_client

In [4]:
#export
dim = 768 #+ onehot.n_dim()
QdrantClient.get_collections = lambda self: [c['name'] for c in self.http.collections_api.get_collections().dict()['result']['collections']]
QdrantClient.collection_len = lambda self, name: self.http.collections_api.get_collection(name).dict()['result']['vectors_count']

In [5]:
#export
prod_client = QdrantClient(host='localhost', port=6335)
dev_client  = QdrantClient(host='localhost', port=6334)
client = prod_client if branch == "prod" else dev_client


In [6]:
prod_client.get_collections(),dev_client.get_collections()

(['food_images'], ['glovo', 'food_images'])

In [7]:
# client.recreate_collection(collection_name,dim,qdrant_client.http.models.Distance.DOT)

In [8]:
collection_name

'food_images'

In [9]:
client.collection_len(collection_name)

104276

In [10]:
!nbdev_build_lib

Converted 00_clipmodel.ipynb.
Converted 00_custom_pandas.ipynb.
Converted 00_paths.ipynb.
Converted 00_progress_check.ipynb.
Converted 00_psql.ipynb.
Converted 00_qdrant.ipynb.
Converted 00_tools.ipynb.
Converted 01_search.ipynb.
Converted inference.ipynb.
Converted stats.ipynb.
